# Importaciones

In [7]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# Evaluamos los outliers del conjunto de datos
Al tratarse de un conjunto de datos muy pequeño, se decide considerar únicamente los percentiles 1 y 99.

In [5]:
# Cargar el dataset
df = pd.read_csv(r'DATA_UNIDO_FINAL_INPUTADOH_ENCODED.csv')

In [6]:
# Crear un DataFrame para almacenar los outliers
outliers = pd.DataFrame()

# Diccionario para almacenar columnas con outliers
columnas_con_outliers = {}

# Identificar las columnas que no son resultado de One Hot Encoding
columnas_no_one_hot = [col for col in df.columns if not set(df[col].dropna().unique()).issubset({0, 1})]

# Calcular outliers para cada columna numérica que no sea One Hot Encoded
for column in df[columnas_no_one_hot].select_dtypes(include=['float64', 'int64']).columns:
    lower_bound = df[column].quantile(0.01)
    upper_bound = df[column].quantile(0.99)
    
    # Identificar los outliers utilizando los límites de los percentiles
    outliers[column] = ((df[column] < lower_bound) | (df[column] > upper_bound))
    
    # Añadir la columna al diccionario si tiene outliers
    if outliers[column].any():
        columnas_con_outliers[column] = df[column][outliers[column]]

# Mostrar el resumen de las columnas con outliers
for col, outliers in columnas_con_outliers.items():
    print(f"Columna '{col}' tiene {len(outliers)} outliers.")

# Mostrar las columnas con outliers
print("\nColumnas que tienen outliers:")
print(list(columnas_con_outliers.keys()))

Columna 'ID' tiene 2 outliers.
Columna '1ESO_C_1EV' tiene 1 outliers.
Columna '1ESO_C_2EV' tiene 1 outliers.
Columna '1ESO_C_EVF' tiene 2 outliers.
Columna '1ESO_EF_1EV' tiene 1 outliers.
Columna '1ESO_EF_EVF' tiene 1 outliers.
Columna '1ESO_I_2EV' tiene 2 outliers.
Columna '1ESO_M_1EV' tiene 1 outliers.
Columna '1ESO_M_EVF' tiene 1 outliers.
Columna '1ESO_MEDIA_1EV' tiene 2 outliers.
Columna '1ESO_MEDIA_2EV' tiene 1 outliers.
Columna '2ESO_EF_2EV' tiene 2 outliers.
Columna '2ESO_EF_EVF' tiene 1 outliers.
Columna '2ESO_I_1EV' tiene 1 outliers.
Columna '2ESO_I_2EV' tiene 1 outliers.
Columna '2ESO_I_EVF' tiene 2 outliers.
Columna '2ESO_M_1EV' tiene 1 outliers.
Columna '2ESO_M_2EV' tiene 1 outliers.
Columna '2ESO_M_EVF' tiene 1 outliers.
Columna '2ESO_MEDIA_1EV' tiene 1 outliers.
Columna '1FPB_MEDIA_1EV' tiene 2 outliers.
Columna '1FPB_MEDIA_2EV' tiene 2 outliers.
Columna '1FPB_MEDIA_EVF' tiene 2 outliers.
Columna '1FPB_P_1EV' tiene 1 outliers.
Columna '1FPB_P_2EV' tiene 2 outliers.
Colum

# Tratamiento de atípicos - Winsorización
El método de winsorización consiste en sustituir los valores extremos de un conjunto de datos por los valores correspondientes a los percentiles previamente seleccionados.

In [15]:
# DataFrame para almacenar los valores winsorizados
df_winsorized = df.copy()

# Aplicar la winsorización usando percentiles
for column in columnas_con_outliers:
    lower_bound = df[column].quantile(0.01)
    upper_bound = df[column].quantile(0.99)
    
    # Imprimir los límites para la verificación
    print(f"Column: {column}, Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    
    # Winsorizar los valores fuera de los límites usando percentiles
    df_winsorized[column] = winsorize(df[column], limits=[0.01, 0.01]) #1% por arriba y por abajo

# Mostrar el DataFrame winsorizado
print("DataFrame winsorizado:\n", df_winsorized)

Column: ID, Lower Bound: 1.3, Upper Bound: 35.7
Column: 1ESO_C_1EV, Lower Bound: 2.0, Upper Bound: 6.699999999999999
Column: 1ESO_C_2EV, Lower Bound: 1.3, Upper Bound: 5.0
Column: 1ESO_C_EVF, Lower Bound: 2.3, Upper Bound: 7.699999999999999
Column: 1ESO_EF_1EV, Lower Bound: 1.3, Upper Bound: 7.0
Column: 1ESO_EF_EVF, Lower Bound: 3.0, Upper Bound: 7.699999999999999
Column: 1ESO_I_2EV, Lower Bound: 1.3, Upper Bound: 6.399999999999999
Column: 1ESO_M_1EV, Lower Bound: 1.0, Upper Bound: 7.699999999999999
Column: 1ESO_M_EVF, Lower Bound: 2.3, Upper Bound: 7.0
Column: 1ESO_MEDIA_1EV, Lower Bound: 2.3, Upper Bound: 5.699999999999999
Column: 1ESO_MEDIA_2EV, Lower Bound: 3.3, Upper Bound: 6.0
Column: 2ESO_EF_2EV, Lower Bound: 2.3, Upper Bound: 8.7
Column: 2ESO_EF_EVF, Lower Bound: 3.6, Upper Bound: 9.0
Column: 2ESO_I_1EV, Lower Bound: 1.0, Upper Bound: 6.699999999999999
Column: 2ESO_I_2EV, Lower Bound: 1.0, Upper Bound: 7.399999999999999
Column: 2ESO_I_EVF, Lower Bound: 1.3, Upper Bound: 6.69999

C:\Users\abrah\anaconda3\lib\site-packages\numpy\lib\function_base.py:4824: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


# Guardado del nuevo dataset sin atípicos en csv como 'DATA_UNIDO_FINAL_INPUTADOH_ENCODED_NOOUTLIERS.csv'

In [16]:
# Guarda el DataFrame en un archivo CSV
archivo_csv = 'DATA_UNIDO_FINAL_INPUTADOH_ENCODED_NOOUTLIERS.csv'
df.to_csv(archivo_csv, index=False)